In [2]:
import torch
import numpy as np

In [3]:
batch = 4
seq_len = 2
dim = 2
a=torch.rand(batch,seq_len,dim)
b=torch.rand(batch,1,dim)
# b = b.repeat(1,seq_len,1)
# res = torch.concat([a,b],dim=-1)
res = a+b
print(a,b,res)
print(res.shape)

tensor([[[0.8375, 0.3225],
         [0.5488, 0.2301]],

        [[0.4722, 0.1360],
         [0.4242, 0.8377]],

        [[0.2532, 0.2531],
         [0.4185, 0.2456]],

        [[0.8839, 0.7750],
         [0.8140, 0.5263]]]) tensor([[[0.9472, 0.9957]],

        [[0.1605, 0.6259]],

        [[0.4325, 0.5864]],

        [[0.1911, 0.5343]]]) tensor([[[1.7847, 1.3182],
         [1.4960, 1.2258]],

        [[0.6327, 0.7619],
         [0.5846, 1.4636]],

        [[0.6857, 0.8395],
         [0.8510, 0.8320]],

        [[1.0750, 1.3093],
         [1.0051, 1.0607]]])
torch.Size([4, 2, 2])


In [4]:
output = torch.rand(4,5)
target = torch.randint(0,5,(4,1))
pred = output.topk(5, 1, True, True)[1].t()
expand_target = target.view(1, -1).expand_as(pred)
correct = pred.eq(expand_target)

In [5]:
output = torch.rand(4,5)
top1 = output.topk(1)[1]
print(top1[0])

tensor([3])


In [6]:
import random
res = random.sample(range(1000),10)
print(res)

[144, 541, 783, 774, 165, 348, 606, 180, 72, 59]


In [7]:
def sapmle_topk1_prototypes(logits:torch.Tensor, targets:torch.Tensor, topK:int):
    cls_num = logits.shape[1]
    batch = logits.shape[0]
    

    origin_target = []
    new_target = torch.zeros(batch,1).view(-1)
    for i, target in enumerate(targets):
        random_targets = random.sample(range(cls_num),topK+1)
        if target in random_targets:
            new_target[i] = random_targets.index(target)
        else:
            random_targets[0] = target.numpy()
        origin_target =  np.append(origin_target, random_targets)
    
    origin_target = origin_target.reshape(batch,-1)
    new_target = new_target.cuda()

    return new_target, origin_target

In [8]:
cls_num = 10
batch = 4
logits = torch.rand(batch,cls_num)
target = torch.randint(0,cls_num,(batch,1)).view(-1)
new_target, origin_target = sapmle_topk1_prototypes(logits,target,5)

print(logits.topk(5)[1])
print(target)
print(new_target)
print(origin_target)

tensor([[9, 6, 3, 2, 8],
        [5, 7, 4, 1, 8],
        [9, 5, 3, 7, 6],
        [5, 6, 9, 4, 1]])
tensor([8, 6, 2, 5])
tensor([1., 0., 0., 2.], device='cuda:0')
[[3. 8. 6. 4. 1. 9.]
 [6. 3. 2. 4. 7. 5.]
 [2. 4. 9. 5. 8. 0.]
 [9. 2. 5. 4. 1. 8.]]


# .numpy()

In [9]:
a = torch.tensor([1,2])
print(a)
b = a.numpy()
print(b)

tensor([1, 2])
[1 2]


# sample

In [10]:
random.seed(1)
for i in range(2):
    for j in range(2):
        res = random.sample(range(10),5)
        print(res)

[2, 1, 4, 0, 3]
[7, 9, 6, 8, 1]
[1, 7, 0, 6, 3]
[6, 0, 7, 2, 5]


# att rank

In [11]:
batch = 1
k = 2
y = abs(torch.rand(batch, k))
res = abs(torch.rand(batch, k))
print(y)
print(res)

tensor([[0.9217, 0.7232]])
tensor([[0.0415, 0.9692]])


In [12]:
def att_rank(y):
    exp_y = torch.exp(y)
    # print(f'exp_y:{exp_y}')
    sum = torch.sum(exp_y,dim=-1).unsqueeze(-1)
    # print(f'sum:{sum}')
    y = exp_y / sum
    return y

In [13]:
y = att_rank(y)
res = att_rank(res)
loss1 = y*torch.log(res)
loss2 = (1-y)*torch.log(1-res)
Loss = loss1+loss2
print(Loss.sum())

tensor(-1.6860)


# relu

In [14]:
import torch.nn.functional as F

x = torch.tensor([-1,1])
print(x)
print(F.relu(x))

tensor([-1,  1])
tensor([0, 1])


In [15]:
cls_num = 10
batch = 4
k = 5
logits = torch.rand(batch,cls_num)
print(f'logits:{logits}')
indices = torch.randint(0,cls_num,(batch,k))
print(f'indices:{indices}')
logits2 = torch.stack([logits[i][indices[i]] for i in range(batch)])
print(logits2)

logits:tensor([[0.7322, 0.7518, 0.5606, 0.7673, 0.2505, 0.3466, 0.2546, 0.4729, 0.4775,
         0.0329],
        [0.0604, 0.7217, 0.2045, 0.5660, 0.2487, 0.7841, 0.5745, 0.1031, 0.1405,
         0.0032],
        [0.7634, 0.6003, 0.2764, 0.4705, 0.7311, 0.0862, 0.1134, 0.0888, 0.1834,
         0.6933],
        [0.7720, 0.9737, 0.1762, 0.4136, 0.4142, 0.2692, 0.9401, 0.9956, 0.7636,
         0.0167]])
indices:tensor([[6, 9, 6, 4, 9],
        [3, 9, 2, 1, 3],
        [0, 6, 4, 8, 5],
        [3, 9, 0, 4, 6]])
tensor([[0.2546, 0.0329, 0.2546, 0.2505, 0.0329],
        [0.5660, 0.0032, 0.2045, 0.7217, 0.5660],
        [0.7634, 0.1134, 0.7311, 0.1834, 0.0862],
        [0.4136, 0.0167, 0.7720, 0.4142, 0.9401]])


In [31]:
def one_hot(y, cls_num):
    one_hot_target = torch.zeros((y.shape[0], cls_num))
    for i,target in enumerate(y):
            one_hot_target[i][target] = 1
    return one_hot_target

In [32]:
a = torch.tensor([1,2,3])
res = one_hot(a,100)
print(res)

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         